# Set Up

### Import the libraries

In [1]:
# Import LUSID
import lusid.models as models
import lusid_sample_data as import_data

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint
import pandas as pd
import json
import lusid
import uuid

# Authenticate our user and create our API client
api_factory = import_data.authenticate_secrets()


print ('LUSID Environment Initialised')
print ('LUSID API Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID API Version:  0.5.3640.0


![Init](img/fundoperations-lusidinit.gif)

### Create a scope

In [2]:
# Generate a unique id for your scope
scope_id = import_data.create_scope_id()
# Set the name of your scope
scope = 'fund-operations-{}'.format(scope_id)
prettyprint.heading('Scope', scope)

Scope: fund-operations-37f8-d2d8-4050-74


![Scope](img/fundoperations-scope.gif)

### Create properties to hold important information

In [3]:
property_codes = ["SubscriptionType", "InvestorId", "FundCode", "OperationStage"]
property_keys = {}

for property_code in property_codes:
    
    # Create our request to define a new property
    request = models.CreatePropertyDefinitionRequest(
        domain='Transaction',
        scope=scope,
        code=property_code,
        value_required=False,
        display_name=property_code,
        data_type_id=models.ResourceId(scope='system', code='string'))

    # Call LUSID to create our new property
    response = api_factory.build(lusid.api.PropertyDefinitionsApi).create_property_definition(
        definition=request)

    # Grab the key off the response to use when referencing this property in other LUSID calls
    property_keys[property_code] = response.key

    # Pretty print our key
    prettyprint.heading(f'{property_code} Property Key', response.key)

SubscriptionType Property Key: Transaction/fund-operations-37f8-d2d8-4050-74/SubscriptionType
InvestorId Property Key: Transaction/fund-operations-37f8-d2d8-4050-74/InvestorId
FundCode Property Key: Transaction/fund-operations-37f8-d2d8-4050-74/FundCode
OperationStage Property Key: Transaction/fund-operations-37f8-d2d8-4050-74/OperationStage


![Properties](img/fundoperations-properties.gif)

### Create the investor's portfolio

In [4]:
# Set the code of your portfolio
investor_portfolio_code = 'Investor1'

# Set the creation date of your portfolio 
portfolio_creation_date = datetime.now(pytz.UTC) - timedelta(days=1052)

# Build your request to create your portfolio
request = models.CreateTransactionPortfolioRequest(
    display_name=investor_portfolio_code,
    code=investor_portfolio_code,
    base_currency='AUD',
    description=None,
    created=portfolio_creation_date,
    corporate_action_source_id=None,
    accounting_method='AverageCost',
    sub_holding_keys=[property_keys["SubscriptionType"], property_keys["FundCode"]],
    properties=None)

# Call LUSID to create your portfolio
response = api_factory.build(lusid.api.TransactionPortfoliosApi).create_portfolio(
    scope=scope,
    transaction_portfolio=request)

# Pretty print the response
prettyprint.portfolio_response(response)

Portfolio Created
Scope: fund-operations-37f8-d2d8-4050-74
Code: Investor1
Portfolio Effective From: 2017-01-16 14:46:58.068780+00:00
Portfolio Created On: 2019-12-04 14:46:58.173238+00:00



![Investor1](img/fundoperations-investor1.gif)

### Create the fund portfolio

In [5]:
# Set the code of your portfolio
fund_portfolio_code = 'Pacfic-Infrastructure-Fund'

# Set the creation date of your portfolio 
portfolio_creation_date = datetime.now(pytz.UTC) - timedelta(days=1052)

# Build your request to create your portfolio
request = models.CreateTransactionPortfolioRequest(
    display_name=fund_portfolio_code,
    code=fund_portfolio_code,
    base_currency='AUD',
    description=None,
    created=portfolio_creation_date,
    corporate_action_source_id=None,
    accounting_method='AverageCost',
    sub_holding_keys=[property_keys["SubscriptionType"], property_keys["InvestorId"]],
    properties=None)

# Call LUSID to create your portfolio
response = api_factory.build(lusid.api.TransactionPortfoliosApi).create_portfolio(
    scope=scope,
    transaction_portfolio=request)

# Pretty print the response
prettyprint.portfolio_response(response)

Portfolio Created
Scope: fund-operations-37f8-d2d8-4050-74
Code: Pacfic-Infrastructure-Fund
Portfolio Effective From: 2017-01-16 14:46:58.222610+00:00
Portfolio Created On: 2019-12-04 14:46:58.340715+00:00



![Fund](img/fundoperations-fund.gif)

### Set up the default transaction mapping

In [6]:
# Import the default transaction type configuration and load it into a dictionary 
default_transaction_mapping=open('data/default_transaction_mapping.json').read()
default_transaction_mapping = json.loads(default_transaction_mapping)
# Pretty print your configuration
pprint.pprint(default_transaction_mapping)

{'transactionConfigRequests': [{'aliases': [{'description': 'Purchase',
                                             'transactionClass': 'Basic',
                                             'transactionGroup': 'default',
                                             'transactionRoles': 'LongLonger',
                                             'type': 'Buy'},
                                            {'description': 'PURCHASE',
                                             'transactionClass': 'Basic',
                                             'transactionGroup': 'alt1',
                                             'transactionRoles': 'LongLonger',
                                             'type': 'BY'}],
                                'movements': [{'direction': 1,
                                               'mappings': [],
                                               'movementTypes': 'StockMovement',
                                               'properties': [],
       

In [7]:
# Initialise your list of configuration requests, one for each transaction type
configuration_requests = []

# Iterate over your configurations in the default mapping
for configuration in default_transaction_mapping['transactionConfigRequests']:
    
    # Initialise your list of aliases for this configuration
    aliases = []
    
    # Iterate over the aliases in the imported config 
    for alias in configuration['aliases']:
        # Append the alias to your list
        aliases.append(
            models.TransactionConfigurationTypeAlias(
                type=alias['type'],
                description=alias['description'],
                transaction_class=alias['transactionClass'],
                transaction_group=alias['transactionGroup'],
                transaction_roles=alias['transactionRoles']))
        
        
    # Initialise your list of movements for this configuration
    movements = []
    
    # Iterate over the movements in the impoted config
    for movement in configuration['movements']:
        
        # Add properties if they exist in the config
        if len (movement['properties']) > 0:
            key = movement['properties'][0]['key']
            value = models.PropertyValue(label_value=movement['properties'][0]['value'])
            properties={key: models.PerpetualProperty(
                key=key,
                value=value)}                                    
        else:
            properties=None
            
        # Append the movement to your list
        movements.append(
            models.TransactionConfigurationMovementDataRequest(
                movement_types=movement['movementTypes'],
                side=movement['side'],
                direction=movement['direction'],
                properties=properties,
                mappings=None))
    
    # Build your configuration for this transaction type
    configuration_requests.append(
        models.TransactionConfigurationDataRequest(
        aliases=aliases,
        movements=movements,
        properties=None))

# Call LUSID to set your configuration for our transaction types
default_transaction_mapping['transactionConfigRequests'] = configuration_requests
response = api_factory.build(lusid.api.SystemConfigurationApi).set_configuration_transaction_types(
    types=default_transaction_mapping)

# Pretty print the response
prettyprint.transaction_type_response(response)

Transaction Configuration #1

Transaction Type Aliases
Transaction Type: Buy
Alias Description: Purchase
Transaction Class: Basic
Transaction Group: default
Transaction Roles: LongLonger


Transaction Type: BY
Alias Description: PURCHASE
Transaction Class: Basic
Transaction Group: alt1
Transaction Roles: LongLonger


Transaction Movements
Movement Types: StockMovement
Side: Side1
Direction: 1
Movement Types: CashCommitment
Side: Side2
Direction: -1



Transaction Configuration #2

Transaction Type Aliases
Transaction Type: Sell
Alias Description: Sale
Transaction Class: Basic
Transaction Group: default
Transaction Roles: LongShorter


Transaction Type: SL
Alias Description: SALE
Transaction Class: Basic
Transaction Group: alt1
Transaction Roles: LongShorter


Transaction Movements
Movement Types: StockMovement
Side: Side1
Direction: -1
Movement Types: CashCommitment
Side: Side2
Direction: 1



Transaction Configuration #3

Transaction Type Aliases
Transaction Type: CoverShort
Alias Des

![Default Transaction Mapping](img/fundoperations-transactiontypeconfig.gif)

## Add transaction types for Fund Operations

### Commitment

In [8]:
# Call LUSID to create your transaction type
response = api_factory.build(lusid.api.SystemConfigurationApi).create_configuration_transaction_type(
    type = models.TransactionConfigurationDataRequest(
        aliases=[
            models.TransactionConfigurationTypeAlias(
                type='Commitment',
                description='The commitment of funds by an investor',
                transaction_class='FundOperations',
                transaction_group='FundOperations-Investor',
                transaction_roles='Longer')
        ],
        movements=[
            models.TransactionConfigurationMovementDataRequest(
                movement_types='CashCommitment',
                side='Side2',
                direction=1,
                properties=None,
                mappings=[
                    models.TransactionPropertyMappingRequest(
                        property_key=property_keys["SubscriptionType"],
                        set_to="Committed")
                ]
            )
        ],
        properties=None))

# Call LUSID to create your transaction type
response = api_factory.build(lusid.api.SystemConfigurationApi).create_configuration_transaction_type(
    type = models.TransactionConfigurationDataRequest(
        aliases=[
            models.TransactionConfigurationTypeAlias(
                type='Commitment',
                description='The commitment of funds by an investor',
                transaction_class='FundOperations',
                transaction_group='FundOperations-Fund',
                transaction_roles='Longer')
        ],
        movements=[
            models.TransactionConfigurationMovementDataRequest(
                movement_types='CashCommitment',
                side='Side2',
                direction=-1,
                properties=None,
                mappings=[
                    models.TransactionPropertyMappingRequest(
                        property_key=property_keys["SubscriptionType"],
                        set_to="Committed")
                ]),
            models.TransactionConfigurationMovementDataRequest(
                movement_types='CashCommitment',
                side='Side2',
                direction=1,
                properties=None,
                mappings=[
                    models.TransactionPropertyMappingRequest(
                        property_key=property_keys["SubscriptionType"],
                        set_to="Committed"),
                    models.TransactionPropertyMappingRequest(
                        property_key=property_keys["InvestorId"],
                        set_to="")
                ]
            )
        ],
        properties=None))

prettyprint.transaction_type_response(response, filters=['Commitment'])

Transaction Configuration #20

Transaction Type Aliases
Transaction Type: Commitment
Alias Description: The commitment of funds by an investor
Transaction Class: FundOperations
Transaction Group: FundOperations-Investor
Transaction Roles: Longer


Transaction Movements
Movement Types: CashCommitment
Side: Side2
Direction: 1



Transaction Configuration #21

Transaction Type Aliases
Transaction Type: Commitment
Alias Description: The commitment of funds by an investor
Transaction Class: FundOperations
Transaction Group: FundOperations-Fund
Transaction Roles: Longer


Transaction Movements
Movement Types: CashCommitment
Side: Side2
Direction: -1
Movement Types: CashCommitment
Side: Side2
Direction: 1





![Add Transaction Mapping](img/fundoperations-addtype.gif)

### Capital call

In [9]:
# Call LUSID to create your transaction type
response = api_factory.build(lusid.api.SystemConfigurationApi).create_configuration_transaction_type(
    type = models.TransactionConfigurationDataRequest(
        aliases=[
            models.TransactionConfigurationTypeAlias(
                type='Capital call',
                description='The provision of funds by an investor in response to a capital call',
                transaction_class='FundOperations',
                transaction_group='FundOperations-Investor',
                transaction_roles='LongShorter')
        ],
        movements=[
            models.TransactionConfigurationMovementDataRequest(
                movement_types='CashSettlement',
                side='Side2',
                direction=-1,
                properties=None,
                mappings=[
                    models.TransactionPropertyMappingRequest(
                        property_key=property_keys["SubscriptionType"],
                        set_to="Committed")
                ]),
            models.TransactionConfigurationMovementDataRequest(
                movement_types='CashCommitment',
                side='Side2',
                direction=1,
                properties=None,
                mappings=[
                    models.TransactionPropertyMappingRequest(
                        property_key=property_keys["SubscriptionType"],
                        set_to="Capital")
                ]
            )
        ],
        properties=None))

# Call LUSID to create your transaction type
response = api_factory.build(lusid.api.SystemConfigurationApi).create_configuration_transaction_type(
    type = models.TransactionConfigurationDataRequest(
        aliases=[
            models.TransactionConfigurationTypeAlias(
                type='Capital call',
                description='The provision of funds by an investor in response to a capital call',
                transaction_class='FundOperations',
                transaction_group='FundOperations-Fund',
                transaction_roles='Longer')
        ],
        movements=[
            models.TransactionConfigurationMovementDataRequest(
                movement_types='CashSettlement',
                side='Side2',
                direction=1,
                properties=None,
                mappings=[
                    models.TransactionPropertyMappingRequest(
                        property_key=property_keys["SubscriptionType"],
                        set_to="Committed")
                ]),
             models.TransactionConfigurationMovementDataRequest(
                movement_types='CashSettlement',
                side='Side2',
                direction=-1,
                properties=None,
                mappings=[
                    models.TransactionPropertyMappingRequest(
                        property_key=property_keys["SubscriptionType"],
                        set_to="Capital")
                ]),
            models.TransactionConfigurationMovementDataRequest(
                movement_types='CashSettlement',
                side='Side2',
                direction=-1,
                properties=None,
                mappings=[
                    models.TransactionPropertyMappingRequest(
                        property_key=property_keys["SubscriptionType"],
                        set_to="Committed"),
                    models.TransactionPropertyMappingRequest(
                        property_key=property_keys["InvestorId"],
                        set_to="")
                ]),
            models.TransactionConfigurationMovementDataRequest(
                movement_types='CashSettlement',
                side='Side2',
                direction=1,
                properties=None,
                mappings=[
                    models.TransactionPropertyMappingRequest(
                        property_key=property_keys["SubscriptionType"],
                        set_to="Capital"),
                    models.TransactionPropertyMappingRequest(
                        property_key=property_keys["InvestorId"],
                        set_to="")
                ]
            )
        ],
        properties=None))

prettyprint.transaction_type_response(response, filters=['Capital call'])

Transaction Configuration #22

Transaction Type Aliases
Transaction Type: Capital call
Alias Description: The provision of funds by an investor in response to a capital call
Transaction Class: FundOperations
Transaction Group: FundOperations-Investor
Transaction Roles: LongShorter


Transaction Movements
Movement Types: CashSettlement
Side: Side2
Direction: -1
Movement Types: CashCommitment
Side: Side2
Direction: 1



Transaction Configuration #23

Transaction Type Aliases
Transaction Type: Capital call
Alias Description: The provision of funds by an investor in response to a capital call
Transaction Class: FundOperations
Transaction Group: FundOperations-Fund
Transaction Roles: Longer


Transaction Movements
Movement Types: CashSettlement
Side: Side2
Direction: 1
Movement Types: CashSettlement
Side: Side2
Direction: -1
Movement Types: CashSettlement
Side: Side2
Direction: -1
Movement Types: CashSettlement
Side: Side2
Direction: 1





### Distribution

In [10]:
# Call LUSID to create your transaction type
response = api_factory.build(lusid.api.SystemConfigurationApi).create_configuration_transaction_type(
    type = models.TransactionConfigurationDataRequest(
        aliases=[
            models.TransactionConfigurationTypeAlias(
                type='Distribution',
                description='Income distributed from a fund',
                transaction_class='FundOperations',
                transaction_group='FundOperations-Investor',
                transaction_roles='Longer')
        ],
        movements=[
            models.TransactionConfigurationMovementDataRequest(
                movement_types='CashCommitment',
                side='Side2',
                direction=-1,
                properties=None,
                mappings=[
                    models.TransactionPropertyMappingRequest(
                        property_key=property_keys["SubscriptionType"],
                        set_to="Income")
                ]
            )
        ],
        properties=None))


# Call LUSID to create your transaction type
response = api_factory.build(lusid.api.SystemConfigurationApi).create_configuration_transaction_type(
    type = models.TransactionConfigurationDataRequest(
        aliases=[
            models.TransactionConfigurationTypeAlias(
                type='Distribution',
                description='Income distributed from a fund',
                transaction_class='FundOperations',
                transaction_group='FundOperations-Fund',
                transaction_roles='Shorter')
        ],
        movements=[
            models.TransactionConfigurationMovementDataRequest(
                movement_types='CashCommitment',
                side='Side2',
                direction=-1,
                properties=None,
                mappings=[
                    models.TransactionPropertyMappingRequest(
                        property_key=property_keys["SubscriptionType"],
                        set_to="Income"),
                    models.TransactionPropertyMappingRequest(
                        property_key=property_keys["InvestorId"],
                        set_to="")
                ]
            ),
            models.TransactionConfigurationMovementDataRequest(
                movement_types='CashCommitment',
                side='Side2',
                direction=1,
                properties=None,
                mappings=[
                    models.TransactionPropertyMappingRequest(
                        property_key=property_keys["SubscriptionType"],
                        set_to="Income")
                ]
            )
        ],
        properties=None))


prettyprint.transaction_type_response(response, filters=['Distribution'])

Transaction Configuration #24

Transaction Type Aliases
Transaction Type: Distribution
Alias Description: Income distributed from a fund
Transaction Class: FundOperations
Transaction Group: FundOperations-Investor
Transaction Roles: Longer


Transaction Movements
Movement Types: CashCommitment
Side: Side2
Direction: -1



Transaction Configuration #25

Transaction Type Aliases
Transaction Type: Distribution
Alias Description: Income distributed from a fund
Transaction Class: FundOperations
Transaction Group: FundOperations-Fund
Transaction Roles: Shorter


Transaction Movements
Movement Types: CashCommitment
Side: Side2
Direction: -1
Movement Types: CashCommitment
Side: Side2
Direction: 1





## Add transaction types for Asset Transactions

### Initial investment

In [11]:
# Call LUSID to create your transaction type
response = api_factory.build(lusid.api.SystemConfigurationApi).create_configuration_transaction_type(
    type = models.TransactionConfigurationDataRequest(
        aliases=[
            models.TransactionConfigurationTypeAlias(
                type='Initial investment',
                description='The purchase of a security',
                transaction_class='AssetTransactions',
                transaction_group='AssetTransactions',
                transaction_roles='Longer')
        ],
        movements=[
            models.TransactionConfigurationMovementDataRequest(
                movement_types='StockMovement',
                side='Side1',
                direction=1,
                properties=None,
                mappings=[
                    models.TransactionPropertyMappingRequest(
                        property_key=property_keys["InvestorId"],
                        set_to=""),
                    models.TransactionPropertyMappingRequest(
                        property_key=property_keys["SubscriptionType"],
                        set_to="Capital")
                ]
            ),
            models.TransactionConfigurationMovementDataRequest(
                movement_types='CashCommitment',
                side='Side2',
                direction=-1,
                properties=None,
                mappings=[
                    models.TransactionPropertyMappingRequest(
                        property_key=property_keys["SubscriptionType"],
                        set_to="Capital"),
                    models.TransactionPropertyMappingRequest(
                        property_key=property_keys["InvestorId"],
                        set_to="")
                ]
            )
        ],
        properties=None))

prettyprint.transaction_type_response(response, filters=['Initial investment'])

Transaction Configuration #26

Transaction Type Aliases
Transaction Type: Initial investment
Alias Description: The purchase of a security
Transaction Class: AssetTransactions
Transaction Group: AssetTransactions
Transaction Roles: Longer


Transaction Movements
Movement Types: StockMovement
Side: Side1
Direction: 1
Movement Types: CashCommitment
Side: Side2
Direction: -1





### Distribution

In [12]:
# Call LUSID to create your transaction type
response = api_factory.build(lusid.api.SystemConfigurationApi).create_configuration_transaction_type(
    type = models.TransactionConfigurationDataRequest(
        aliases=[
            models.TransactionConfigurationTypeAlias(
                type='Distribution',
                description='Income distributed from an infrastructure or real asset',
                transaction_class='AssetTransactions',
                transaction_group='AssetTransactions',
                transaction_roles='Longer')
        ],
        movements=[
            models.TransactionConfigurationMovementDataRequest(
                movement_types='CashCommitment',
                side='Side2',
                direction=1,
                properties=None,
                mappings=[
                    models.TransactionPropertyMappingRequest(
                        property_key=property_keys["SubscriptionType"],
                        set_to="Income"),
                    models.TransactionPropertyMappingRequest(
                        property_key=property_keys["InvestorId"],
                        set_to="")
                ]
            )
        ],
        properties=None))

prettyprint.transaction_type_response(response, filters=['Distribution'])

Transaction Configuration #24

Transaction Type Aliases
Transaction Type: Distribution
Alias Description: Income distributed from a fund
Transaction Class: FundOperations
Transaction Group: FundOperations-Investor
Transaction Roles: Longer


Transaction Movements
Movement Types: CashCommitment
Side: Side2
Direction: -1



Transaction Configuration #25

Transaction Type Aliases
Transaction Type: Distribution
Alias Description: Income distributed from a fund
Transaction Class: FundOperations
Transaction Group: FundOperations-Fund
Transaction Roles: Shorter


Transaction Movements
Movement Types: CashCommitment
Side: Side2
Direction: -1
Movement Types: CashCommitment
Side: Side2
Direction: 1



Transaction Configuration #27

Transaction Type Aliases
Transaction Type: Distribution
Alias Description: Income distributed from an infrastructure or real asset
Transaction Class: AssetTransactions
Transaction Group: AssetTransactions
Transaction Roles: Longer


Transaction Movements
Movement Type

![Final Transaction Mapping](img/fundoperations-finaltypes.gif)

### Set up dates to work with

In [13]:
current_time = datetime.now(pytz.UTC)
day_1 = current_time - timedelta(days=6)
day_2 = current_time - timedelta(days=5)
day_3 = current_time - timedelta(days=4)
day_4 = current_time - timedelta(days=3)
day_5 = current_time - timedelta(days=2)
day_6 = current_time - timedelta(days=1)

### Create a function to make fund operation transactions

In [14]:
def subscription_transaction(date, units, transaction_type, investor_portfolio_code, fund_portfolio_code, status=None):
    """
    This function creates subscription transactions for committing to a fund as well as provisioning capital via
    a capital call. 
    
    param date (datetime): The datetime to upsert the transactions at
    param units (float): The number of units in the transaction
    param transaction_type (str): The code for the transaction type for this subscription transaction
    param investor_portfolio_code (str): The code for the investor's portfolio
    param fund_portfolio_code (str): The code for the fund portfolio
    
    return unique_transaction_id (str): The unique id for the transactions
    """
    
    transactions = {}
    
    unique_transaction_id = str(uuid.uuid4())
    
    investor_properties = {
        property_keys['FundCode']: models.ModelProperty(
            key=property_keys['FundCode'],
            value=models.PropertyValue(
                label_value=fund_portfolio_code)
        )
    }
    
    if type(status) is not type(None):
        investor_properties[property_keys['OperationStage']] = models.ModelProperty(
            key=property_keys['OperationStage'],
            value=models.PropertyValue(
                label_value=status)
        )
            
            
    # Construct the transaction for the investor's portfolio
    transactions[investor_portfolio_code] = models.TransactionRequest(
        transaction_id=unique_transaction_id,
        type=transaction_type,
        instrument_identifiers={
            'Instrument/default/Currency': 'AUD'
        },
        transaction_date=date,
        settlement_date=date,
        units=units,
        transaction_price=models.TransactionPrice(
            price=1,
            type='Price'),
        total_consideration=models.CurrencyAndAmount(
            amount=units,
            currency='AUD'),
        source='FundOperations-Investor',
        transaction_currency='AUD',
        properties=investor_properties)
    
    # Construct the transaction for the fund's portfolio
    transactions[fund_portfolio_code] = models.TransactionRequest(
        transaction_id=unique_transaction_id,
        type=transaction_type,
        instrument_identifiers={
            'Instrument/default/Currency': 'AUD'
        },
        transaction_date=date,
        settlement_date=date,
        units=units,
        transaction_price=models.TransactionPrice(
            price=1,
            type='Price'),
        total_consideration=models.CurrencyAndAmount(
            amount=units,
            currency='AUD'),
        source='FundOperations-Fund',
        transaction_currency='AUD',
        properties={
            property_keys['InvestorId'] : models.ModelProperty(
                key=property_keys['InvestorId'],
                value=models.PropertyValue(
                    label_value=investor_portfolio_code)
            )
        })

    # Loop over the portfolios
    for portfolio_code, transaction in transactions.items():
        
        # Call LUSID to upsert the transactions
        response = api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_transactions(
            scope=scope,
            code=portfolio_code,
            transactions=[transaction])

        # Print the response from LUSID using pretty formatting 
        prettyprint.transactions_response(
            response,
            scope,
            portfolio_code)
    
    return unique_transaction_id

# Day 1 - Investor commits AUD10,000,000 to the fund

### Make the commitment transaction

In [15]:
subscription_transaction(
    date=day_1, 
    units=10000000, 
    transaction_type='Commitment',
    investor_portfolio_code=investor_portfolio_code,
    fund_portfolio_code=fund_portfolio_code)

Transactions Successfully Upserted into Portfolio
Scope: fund-operations-37f8-d2d8-4050-74
Code: Investor1
Transactions Effective From: 2019-11-28 14:47:00.209842+00:00
Transactions Created On: 2019-12-04 14:47:00.461500+00:00

Transactions Successfully Upserted into Portfolio
Scope: fund-operations-37f8-d2d8-4050-74
Code: Pacfic-Infrastructure-Fund
Transactions Effective From: 2019-11-28 14:47:00.209842+00:00
Transactions Created On: 2019-12-04 14:47:00.708719+00:00



'059f7189-25cc-477c-a9b8-aee70cc4dc76'

![Comittment Transaction](img/fundoperations-day1commitment.gif)

### Get the holdings of the investor's portfolio

In [16]:
holdings = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
    scope=scope,
    code=investor_portfolio_code,
    property_keys=['Instrument/default/Name'])

prettyprint.sub_holdings(holdings)

Lusid Unique Instrument Id       Units Transaction/SubscriptionType  \
0                    CCY_AUD 10000000.00                    Committed   

         Transaction/FundCode Instrument_Name  Total Cost Currency  
0  Pacfic-Infrastructure-Fund         CCY_AUD 10000000.00      AUD

![Comittment Holdings](img/fundoperations-day1holding.gif)

### Get the holdings of the fund portfolio

In [17]:
holdings = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
    scope=scope,
    code=fund_portfolio_code,
    property_keys=['Instrument/default/Name'])

prettyprint.sub_holdings(holdings)

Lusid Unique Instrument Id        Units Transaction/SubscriptionType  \
0                    CCY_AUD -10000000.00                    Committed   
1                    CCY_AUD  10000000.00                    Committed   

  Transaction/InvestorId Instrument_Name   Total Cost Currency  
0              Investor1         CCY_AUD -10000000.00      AUD  
1                                CCY_AUD  10000000.00      AUD

# Day 2 - The fund makes a capital call of AUD5,000,000 to the investor

### The investor responds to the capital call and provides the funds

In [18]:
subscription_transaction(
    date=day_2, 
    units=5000000, 
    transaction_type='Capital call',
    investor_portfolio_code=investor_portfolio_code,
    fund_portfolio_code=fund_portfolio_code)

Transactions Successfully Upserted into Portfolio
Scope: fund-operations-37f8-d2d8-4050-74
Code: Investor1
Transactions Effective From: 2019-11-29 14:47:00.209842+00:00
Transactions Created On: 2019-12-04 14:47:01.228333+00:00

Transactions Successfully Upserted into Portfolio
Scope: fund-operations-37f8-d2d8-4050-74
Code: Pacfic-Infrastructure-Fund
Transactions Effective From: 2019-11-29 14:47:00.209842+00:00
Transactions Created On: 2019-12-04 14:47:01.466310+00:00



'c2f9a222-cb49-445d-abbc-c5e483e47267'

### See the updated investor's holdings

In [19]:
holdings = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
    scope=scope,
    code=investor_portfolio_code,
    property_keys=['Instrument/default/Name'])

prettyprint.sub_holdings(holdings)

Lusid Unique Instrument Id      Units Transaction/SubscriptionType  \
0                    CCY_AUD 5000000.00                    Committed   
1                    CCY_AUD 5000000.00                      Capital   

         Transaction/FundCode Instrument_Name  Total Cost Currency  
0  Pacfic-Infrastructure-Fund         CCY_AUD  5000000.00      AUD  
1  Pacfic-Infrastructure-Fund         CCY_AUD  5000000.00      AUD

![Capital Call Holdings](img/fundoperations-day2holding.gif)

### The fund tracks this response

In [20]:
holdings = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
    scope=scope,
    code=fund_portfolio_code,
    property_keys=['Instrument/default/Name'])

prettyprint.sub_holdings(holdings)

Lusid Unique Instrument Id       Units Transaction/InvestorId  \
0                    CCY_AUD -5000000.00              Investor1   
1                    CCY_AUD  5000000.00                          
2                    CCY_AUD -5000000.00              Investor1   
3                    CCY_AUD  5000000.00                          

  Transaction/SubscriptionType Instrument_Name  Total Cost Currency  
0                    Committed         CCY_AUD -5000000.00      AUD  
1                    Committed         CCY_AUD  5000000.00      AUD  
2                      Capital         CCY_AUD -5000000.00      AUD  
3                      Capital         CCY_AUD  5000000.00      AUD

# Day 3 - The fund purchases an alternative physical asset

### Add the alternative asset to the LUSID instrument master

In [21]:
# Create the bespoke hedge contract definition
shopping_centre_asset = {
    "identifier": "SYD_241232",
    "no_units": 76,
    "average_retail_lease": "2.3 years",
    "term": "5yr",
    "freq": "Quarterly",
    "rating": "B1"
}

# Create the definition for your instrument, attaching the bespoke contract
shopping_centre_asset_instrument = models.InstrumentDefinition(
    name='Sydney_Bondi_Junc_Westfield',
    identifiers={
        'ClientInternal': models.InstrumentIdValue(
            value=shopping_centre_asset["identifier"])},
    definition=models.InstrumentEconomicDefinition(
        instrument_format='JSON',
        content=json.dumps(shopping_centre_asset)
    )
)

response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments(
    instruments={"shopping_centre": shopping_centre_asset_instrument})

prettyprint.instrument_response(response)

Instruments Successfully Upserted: 


Instrument ClientInternal ID LUSID Instrument ID
0  shopping_centre        SYD_241232       LUID_FD2YQ7EQ

### Purchase the asset

In [22]:
transaction = models.TransactionRequest(
    transaction_id='Purchase_22319041341',
    type='Initial investment',
    instrument_identifiers={
        'Instrument/default/ClientInternal': 'SYD_241232'
    },
    transaction_date=day_3,
    settlement_date=day_3,
    units=1,
    transaction_price=models.TransactionPrice(
        price=5000000,
        type='Price'),
    total_consideration=models.CurrencyAndAmount(
        amount=5000000,
        currency='AUD'),
    source='AssetTransactions',
    transaction_currency='AUD')

# Call LUSID to upsert your transactions for this portfolio
response = api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_transactions(
    scope=scope,
    code=fund_portfolio_code,
    transactions=[transaction])

# Print the response from LUSID using pretty formatting 
prettyprint.transactions_response(
    response,
    scope,
    fund_portfolio_code)

Transactions Successfully Upserted into Portfolio
Scope: fund-operations-37f8-d2d8-4050-74
Code: Pacfic-Infrastructure-Fund
Transactions Effective From: 2019-11-30 14:47:00.209842+00:00
Transactions Created On: 2019-12-04 14:47:02.121501+00:00



### See the updated fund holdings

In [23]:
holdings = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
    scope=scope,
    code=fund_portfolio_code,
    property_keys=['Instrument/default/Name'])

prettyprint.sub_holdings(holdings)

Lusid Unique Instrument Id       Units Transaction/SubscriptionType  \
0              LUID_FD2YQ7EQ        1.00                      Capital   
1                    CCY_AUD -5000000.00                    Committed   
2                    CCY_AUD  5000000.00                    Committed   
3                    CCY_AUD -5000000.00                      Capital   

  Transaction/InvestorId              Instrument_Name  Total Cost Currency  
0                         Sydney_Bondi_Junc_Westfield  5000000.00      AUD  
1              Investor1                      CCY_AUD -5000000.00      AUD  
2                                             CCY_AUD  5000000.00      AUD  
3              Investor1                      CCY_AUD -5000000.00      AUD

# Day 4 - Investor 2 Makes AUD2,500,000 committment 

### Create a portfolio for the new investor

In [24]:
# Set the code of your portfolio
investor2_portfolio_code = 'Investor2'

# Set the creation date of your portfolio 
portfolio_creation_date = datetime.now(pytz.UTC) - timedelta(days=1052)

# Build your request to create your portfolio
request = models.CreateTransactionPortfolioRequest(
    display_name=investor2_portfolio_code,
    code=investor2_portfolio_code,
    base_currency='AUD',
    description=None,
    created=portfolio_creation_date,
    corporate_action_source_id=None,
    accounting_method='AverageCost',
    sub_holding_keys=[property_keys["SubscriptionType"], property_keys["FundCode"], property_keys["OperationStage"]],
    properties=None)

# Call LUSID to create your portfolio
response = api_factory.build(lusid.api.TransactionPortfoliosApi).create_portfolio(
    scope=scope,
    transaction_portfolio=request)

# Pretty print the response
prettyprint.portfolio_response(response)

Portfolio Created
Scope: fund-operations-37f8-d2d8-4050-74
Code: Investor2
Portfolio Effective From: 2017-01-16 14:47:02.265907+00:00
Portfolio Created On: 2019-12-04 14:47:02.391503+00:00



### Create a commitment for the investor

In [25]:
transaction_id = subscription_transaction(
    date=day_4, 
    units=2500000, 
    transaction_type='Commitment',
    investor_portfolio_code=investor2_portfolio_code,
    fund_portfolio_code=fund_portfolio_code,
    status='Draft')

Transactions Successfully Upserted into Portfolio
Scope: fund-operations-37f8-d2d8-4050-74
Code: Investor2
Transactions Effective From: 2019-12-01 14:47:00.209842+00:00
Transactions Created On: 2019-12-04 14:47:02.625664+00:00

Transactions Successfully Upserted into Portfolio
Scope: fund-operations-37f8-d2d8-4050-74
Code: Pacfic-Infrastructure-Fund
Transactions Effective From: 2019-12-01 14:47:00.209842+00:00
Transactions Created On: 2019-12-04 14:47:02.826921+00:00



In [26]:
holdings = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
    scope=scope,
    code=investor2_portfolio_code,
    property_keys=['Instrument/default/Name'])

prettyprint.sub_holdings(holdings)

Lusid Unique Instrument Id      Units Transaction/OperationStage  \
0                    CCY_AUD 2500000.00                      Draft   

  Transaction/SubscriptionType        Transaction/FundCode Instrument_Name  \
0                    Committed  Pacfic-Infrastructure-Fund         CCY_AUD   

   Total Cost Currency  
0  2500000.00      AUD

In [27]:
response = api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_transaction_properties(
    scope=scope,
    code=investor2_portfolio_code,
    transaction_id=transaction_id,
    transaction_properties={property_keys['OperationStage']: models.ModelProperty(
        key=property_keys['OperationStage'],
        value=models.PropertyValue(
            label_value='Review')
    )})

prettyprint.add_transaction_property(response)

Added transaction properties asAt: 2019-12-04 14:47:03.328705+00:00


In [28]:
holdings = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
    scope=scope,
    code=investor2_portfolio_code,
    property_keys=['Instrument/default/Name'])

prettyprint.sub_holdings(holdings)

Lusid Unique Instrument Id      Units Transaction/OperationStage  \
0                    CCY_AUD 2500000.00                     Review   

  Transaction/SubscriptionType        Transaction/FundCode Instrument_Name  \
0                    Committed  Pacfic-Infrastructure-Fund         CCY_AUD   

   Total Cost Currency  
0  2500000.00      AUD

In [29]:
response = api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_transaction_properties(
    scope=scope,
    code=investor2_portfolio_code,
    transaction_id=transaction_id,
    transaction_properties={property_keys['OperationStage']: models.ModelProperty(
        key=property_keys['OperationStage'],
        value=models.PropertyValue(
            label_value='Approved')
    )})

prettyprint.add_transaction_property(response)

Added transaction properties asAt: 2019-12-04 14:47:03.670338+00:00


In [30]:
holdings = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
    scope=scope,
    code=investor2_portfolio_code,
    property_keys=['Instrument/default/Name'])

prettyprint.sub_holdings(holdings)

Lusid Unique Instrument Id      Units Transaction/OperationStage  \
0                    CCY_AUD 2500000.00                   Approved   

  Transaction/SubscriptionType        Transaction/FundCode Instrument_Name  \
0                    Committed  Pacfic-Infrastructure-Fund         CCY_AUD   

   Total Cost Currency  
0  2500000.00      AUD

### Issue a capital call

In [31]:
subscription_transaction(
    date=day_4, 
    units=2500000, 
    transaction_type='Capital call',
    investor_portfolio_code=investor2_portfolio_code,
    fund_portfolio_code=fund_portfolio_code,
    status='Approved')

Transactions Successfully Upserted into Portfolio
Scope: fund-operations-37f8-d2d8-4050-74
Code: Investor2
Transactions Effective From: 2019-12-01 14:47:00.209842+00:00
Transactions Created On: 2019-12-04 14:47:04.022356+00:00

Transactions Successfully Upserted into Portfolio
Scope: fund-operations-37f8-d2d8-4050-74
Code: Pacfic-Infrastructure-Fund
Transactions Effective From: 2019-12-01 14:47:00.209842+00:00
Transactions Created On: 2019-12-04 14:47:04.202614+00:00



'86fd2334-3270-452b-9bff-b271fe597b31'

### See the updated fund holdings

In [32]:
holdings = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
    scope=scope,
    code=fund_portfolio_code,
    property_keys=['Instrument/default/Name'])

prettyprint.sub_holdings(holdings)

Lusid Unique Instrument Id       Units Transaction/InvestorId  \
0              LUID_FD2YQ7EQ        1.00                          
1                    CCY_AUD -5000000.00              Investor1   
2                    CCY_AUD  5000000.00                          
3                    CCY_AUD -5000000.00              Investor1   
4                    CCY_AUD  2500000.00                          
5                    CCY_AUD -2500000.00              Investor2   

  Transaction/SubscriptionType              Instrument_Name  Total Cost  \
0                      Capital  Sydney_Bondi_Junc_Westfield  5000000.00   
1                    Committed                      CCY_AUD -5000000.00   
2                    Committed                      CCY_AUD  5000000.00   
3                      Capital                      CCY_AUD -5000000.00   
4                      Capital                      CCY_AUD  2500000.00   
5                      Capital                      CCY_AUD -2500000.00   

  Currency  
0      AUD  
1      AUD  
2      AUD  
3      AUD  
4      AUD  
5      AUD

# Day 5 - Income is generated from the asset

### Log the income transaction

In [33]:
transaction = models.TransactionRequest(
    transaction_id='Income_22319041341',
    type='Distribution',
    instrument_identifiers={
        'Instrument/default/ClientInternal': 'SYD_241232'
    },
    transaction_date=day_5,
    settlement_date=day_5,
    units=1,
    transaction_price=models.TransactionPrice(
        price=50000,
        type='Price'),
    total_consideration=models.CurrencyAndAmount(
        amount=50000,
        currency='AUD'),
    source='AssetTransactions',
    transaction_currency='AUD',
    properties={property_keys['OperationStage']: models.ModelProperty(
        key=property_keys['OperationStage'],
        value=models.PropertyValue(
            label_value='Approved')
    )})

# Call LUSID to upsert your transactions for this portfolio
response = api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_transactions(
    scope=scope,
    code=fund_portfolio_code,
    transactions=[transaction])

# Print the response from LUSID using pretty formatting 
prettyprint.transactions_response(
    response,
    scope,
    fund_portfolio_code)

Transactions Successfully Upserted into Portfolio
Scope: fund-operations-37f8-d2d8-4050-74
Code: Pacfic-Infrastructure-Fund
Transactions Effective From: 2019-12-02 14:47:00.209842+00:00
Transactions Created On: 2019-12-04 14:47:04.598632+00:00



### Get the updated fund holdings

In [34]:
holdings = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
    scope=scope,
    code=fund_portfolio_code,
    property_keys=['Instrument/default/Name'])

prettyprint.sub_holdings(holdings)

Lusid Unique Instrument Id       Units Transaction/InvestorId  \
0              LUID_FD2YQ7EQ        1.00                          
1                    CCY_AUD -5000000.00              Investor1   
2                    CCY_AUD  5000000.00                          
3                    CCY_AUD -5000000.00              Investor1   
4                    CCY_AUD  2500000.00                          
5                    CCY_AUD -2500000.00              Investor2   
6                    CCY_AUD    50000.00                          

  Transaction/SubscriptionType              Instrument_Name  Total Cost  \
0                      Capital  Sydney_Bondi_Junc_Westfield  5000000.00   
1                    Committed                      CCY_AUD -5000000.00   
2                    Committed                      CCY_AUD  5000000.00   
3                      Capital                      CCY_AUD -5000000.00   
4                      Capital                      CCY_AUD  2500000.00   
5                      Capital                      CCY_AUD -2500000.00   
6                       Income                      CCY_AUD    50000.00   

  Currency  
0      AUD  
1      AUD  
2      AUD  
3      AUD  
4      AUD  
5      AUD  
6      AUD

# Day 6 - Income is distributed to investors

In [35]:
subscription_transaction(
    date=day_6, 
    units=25000, 
    transaction_type='Distribution',
    investor_portfolio_code=investor_portfolio_code,
    fund_portfolio_code=fund_portfolio_code,
    status='Approved')

subscription_transaction(
    date=day_6, 
    units=25000, 
    transaction_type='Distribution',
    investor_portfolio_code=investor2_portfolio_code,
    fund_portfolio_code=fund_portfolio_code,
    status='Approved')

Transactions Successfully Upserted into Portfolio
Scope: fund-operations-37f8-d2d8-4050-74
Code: Investor1
Transactions Effective From: 2019-12-03 14:47:00.209842+00:00
Transactions Created On: 2019-12-04 14:47:04.913256+00:00

Transactions Successfully Upserted into Portfolio
Scope: fund-operations-37f8-d2d8-4050-74
Code: Pacfic-Infrastructure-Fund
Transactions Effective From: 2019-12-03 14:47:00.209842+00:00
Transactions Created On: 2019-12-04 14:47:05.134571+00:00

Transactions Successfully Upserted into Portfolio
Scope: fund-operations-37f8-d2d8-4050-74
Code: Investor2
Transactions Effective From: 2019-12-03 14:47:00.209842+00:00
Transactions Created On: 2019-12-04 14:47:05.341194+00:00

Transactions Successfully Upserted into Portfolio
Scope: fund-operations-37f8-d2d8-4050-74
Code: Pacfic-Infrastructure-Fund
Transactions Effective From: 2019-12-03 14:47:00.209842+00:00
Transactions Created On: 2019-12-04 14:47:05.497127+00:00



'd29517dc-2a1e-4e1f-ae7e-a35a5aaa23fe'

In [36]:
holdings = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
    scope=scope,
    code=fund_portfolio_code,
    property_keys=['Instrument/default/Name'])

prettyprint.sub_holdings(holdings)

Lusid Unique Instrument Id       Units Transaction/InvestorId  \
0              LUID_FD2YQ7EQ        1.00                          
1                    CCY_AUD -5000000.00              Investor1   
2                    CCY_AUD  5000000.00                          
3                    CCY_AUD -5000000.00              Investor1   
4                    CCY_AUD  2500000.00                          
5                    CCY_AUD -2500000.00              Investor2   
6                    CCY_AUD    25000.00              Investor1   
7                    CCY_AUD    25000.00              Investor2   

  Transaction/SubscriptionType              Instrument_Name  Total Cost  \
0                      Capital  Sydney_Bondi_Junc_Westfield  5000000.00   
1                    Committed                      CCY_AUD -5000000.00   
2                    Committed                      CCY_AUD  5000000.00   
3                      Capital                      CCY_AUD -5000000.00   
4                      Capital                      CCY_AUD  2500000.00   
5                      Capital                      CCY_AUD -2500000.00   
6                       Income                      CCY_AUD    25000.00   
7                       Income                      CCY_AUD    25000.00   

  Currency  
0      AUD  
1      AUD  
2      AUD  
3      AUD  
4      AUD  
5      AUD  
6      AUD  
7      AUD

In [37]:
holdings = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
    scope=scope,
    code=investor2_portfolio_code,
    property_keys=['Instrument/default/Name'])

prettyprint.sub_holdings(holdings)

Lusid Unique Instrument Id      Units Transaction/SubscriptionType  \
0                    CCY_AUD 2500000.00                      Capital   
1                    CCY_AUD  -25000.00                       Income   

         Transaction/FundCode Transaction/OperationStage Instrument_Name  \
0  Pacfic-Infrastructure-Fund                   Approved         CCY_AUD   
1  Pacfic-Infrastructure-Fund                   Approved         CCY_AUD   

   Total Cost Currency  
0  2500000.00      AUD  
1   -25000.00      AUD